# System Preparation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import requests
import psycopg2
import sqlalchemy as db
from pyproj import Proj
from shapely.geometry import Point
import folium
pd.set_option("display.max_columns", None)
from sqlalchemy import text

# Data Gathering

In [2]:
def Data_bvarta(kode_kota,conn, id_source='6'):
    """Fungsi untuk mendapatkan Toko kelontong berdasarkan admin level kabupaten/kota
    Parameters
    ----------

    kode_kota : list of string
        kode kota dalam level kota/kabupaten
    conn : engine conn
        engine connection to database
    id_source : int
        id_source yang digunakan untuk data dari database
    ----------
    """
    from sqlalchemy import text
    if len(kode_kota)>1:
        sql = text("""
        select A.id_merchant, A.id_brand, A.nama_merchant, A.alamat_merchant, A.phone_merchant,  
               A.kode_desa, A.kode_brand, A.id_desa, A.geom, A.nama_brand, A.nama_sub_kategori, A.nama_kategori
        from (select A.*, B.nama_brand, C.nama_sub_kategori, D.nama_kategori
              from tbl_merchant A
              left join tbl_brand B on B.id_brand = A.id_brand
              left join tbl_sub_kategori C on B.id_sub_kategori = C.id_sub_kategori
              left join tbl_kategori D on D.id_kategori = C.id_kategori
              where A.nama_merchant like '%%Toko Kelontong%%' and A.status = 'T') A
        join (select kode_kota, nama_kota, geom
              from tbl_master_kota_gabungan
              where kode_kota in """+str(tuple(kode_kota))+"""  and id_source = """+id_source+""") E on st_intersects(A.geom,E.geom)
        """)
        df_bvt = gpd.read_postgis(sql, conn, crs = 'epsg:4326')
    else:
        sql = text("""
        select A.id_merchant, A.id_brand, A.nama_merchant, A.alamat_merchant, A.phone_merchant,  
               A.kode_desa, A.kode_brand, A.id_desa, A.geom, A.nama_brand, A.nama_sub_kategori, A.nama_kategori
        from (select A.*, B.nama_brand, C.nama_sub_kategori, D.nama_kategori
              from tbl_merchant A
              left join tbl_brand B on B.id_brand = A.id_brand
              left join tbl_sub_kategori C on B.id_sub_kategori = C.id_sub_kategori
              left join tbl_kategori D on D.id_kategori = C.id_kategori
              where A.nama_merchant like '%%Toko Kelontong%%' and A.status = 'T') A
        join (select kode_kota, nama_kota, geom
              from tbl_master_kota_gabungan
              where kode_kota = '"""+str(kode_kota[0])+"""'  and id_source = """+id_source+""") E on st_intersects(A.geom,E.geom)
        """)
        df_bvt = gpd.read_postgis(sql, conn, crs = 'epsg:4326')
    df_bvt.rename(columns = {'geom':'geometry'}, inplace = True)
    df_bvt = gpd.GeoDataFrame(df_bvt)
    
    return df_bvt

In [3]:
def GRID(kode_kota, conn, id_source = '6', ukuran_grid = '1km'):
    """Fungsi untuk mendapatkan data geom grid menggunakan kode kota/kabupaten
    Parameters
    ----------
    kode_kota : list of string
        kode kota dalam level kota/kabupaten
    conn : engine conn
        engine connection to database
    id_source : int
        id_source yang digunakan untuk data dari database
    ----------
    """
    from sqlalchemy import text
    if len(kode_kota)>1:
        sql = text("""
        select a.index as gid, grid_x, grid_y, grid_size, geom
        from tbl_grid_"""+ukuran_grid+""" a
        inner join (
                      select index 
                      from tbl_gid_to_kode_desa_n 
                      where id_source = """+id_source+""" and 
                      left(kode_desa,4) in """+str(tuple(kode_kota))+""") b on a.index = b.index""")
        grid = gpd.read_postgis(sql, conn, crs='epsg:4326')
        
    else:
        sql = text("""
        select a.index as gid, grid_x, grid_y, grid_size, geom
        from tbl_grid_"""+ukuran_grid+""" a
        inner join (
                      select index 
                      from tbl_gid_to_kode_desa_n 
                      where id_source = """+id_source+""" and 
                      left(kode_desa,4) = '"""+str(kode_kota[0])+"""') b on a.index = b.index""")
        grid = gpd.read_postgis(sql, conn, crs='epsg:4326')
    return grid

# Data Processing

In [4]:
def ckdnearest(gdA, gdB, Proj = 'epsg:6933'):
    """Fungsi untuk mendapatkan jarak dan klasifikasi identical data berdasarkan jarak kurang dari 10 meter
    Parameters
    ----------
    gdA : geopandas.GeoDataFrame()
        geodataframe data Client
    gdA : geopandas.GeoDataFrame()
        geodataframe data Bvarta
    ----------
    """
    
    from scipy.spatial import cKDTree
    from shapely.geometry import Point
    import numpy as np
    import pandas as pd
    
    gdA = gdA.to_crs(Proj)
    gdB = gdB.to_crs(Proj)
    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)
    data = gdf.to_crs('epsg:4326')
    data['keterangan'] = np.where(data['dist']<10,'identical','non-identical')
    
    return data

In [6]:
def grid_dominant(grid, df_client, df_bvarta, unique_id_bvarta, unique_id_client):
    """Fungsi untuk mendapatkan dominansi dataframe pada suatu grid
    Parameters
    ----------
    grid = geopandas.GeoDataFrame()
        geodataframe grid
    df_client : geopandas.GeoDataFrame()
        geodataframe data Client
    df_bvarta : geopandas.GeoDataFrame()
        geodataframe data Bvarta
    unique_id_bvarta : string
        name of unique id colomn in bvarta geodataframe
    unique_id_client : string
        name of unique id colomn in client geodataframe
    ----------
    """
    import geopandas as gpd
    import pandas as pd
    import numpy as np
    
    #sjoin poi terhadap grid
    sjoin_bvarta = gpd.sjoin(df_bvarta, grid[['geom','gid']])
    sjoin_client = gpd.sjoin(df_client, grid[['geom','gid']])
    
    #group jumlah poi berdasarkan gid
    group_bvarta = sjoin_bvarta.groupby('gid')[unique_id_bvarta].nunique().reset_index()
    group_bvarta.rename(columns = {unique_id_bvarta:'Total POI Bvarta'}, inplace = True)
    group_client = sjoin_client.groupby('gid')[unique_id_client].nunique().reset_index()
    group_client.rename(columns = {unique_id_client:'Total POI Client'}, inplace =True)
    
    #merge ke grid
    test = pd.merge(grid, group_bvarta[['gid','Total POI Bvarta']], how='left')
    grid = pd.merge(test, group_client[['gid','Total POI Client']], how='left')
    grid = grid.fillna(0)
    
    #select dominan df
    grid['dominan'] = grid[['Total POI Bvarta','Total POI Client']].idxmax(axis=1)
    grid['dominan'] = grid['dominan'].fillna('No Data')
    
    grid['selisih'] = abs(grid['Total POI Bvarta']-grid['Total POI Client'])
    return grid

# Data Visualisasi

In [44]:
def Visualisasi_POI(grid, df_bvarta, df_client,unique_id_bvarta, unique_id_client):
    """Fungsi untuk mendapatkan visualisasi POI
    Parameters
    ----------
    grid = geopandas.GeoDataFrame()
        geodataframe grid
    df_client : geopandas.GeoDataFrame()
        geodataframe data Client
    df_bvarta : geopandas.GeoDataFrame()
        geodataframe data Bvarta
    unique_id_bvarta : string
        name of unique id colomn in bvarta geodataframe
    unique_id_client : string
        name of unique id colomn in client geodataframe
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    import plotly.express as px
    
    bvarta , client = classification(df_client,df_bvarta,unique_id_bvarta, unique_id_client)
    
    merge = pd.concat([bvarta[['id','geometry','keterangan']],
                     client[['id','geometry','keterangan']]])
    #visualisasi
    m = grid.explore(tiles = 'Cartodb Positron',
                     style_kwds = dict(color = 'grey', weight = 0.5),
                     legend = False,
                     popup = False
                    )
    merge.explore(m=m, 
                  column = 'keterangan',
                  cmap = 'GnBu', 
                  marker_kwds=dict(radius = 0.8))
    return m

def Visualisasi_Identical_Location(grid, df_client,df_bvarta,unique_id_bvarta, unique_id_client):
    """Fungsi untuk mendapatkan visualisasi identical POI
    Parameters
    ----------
    grid = geopandas.GeoDataFrame()
        geodataframe grid
    df_client : geopandas.GeoDataFrame()
        geodataframe data Client
    df_bvarta : geopandas.GeoDataFrame()
        geodataframe data Bvarta
    unique_id_bvarta : string
        name of unique id colomn in bvarta geodataframe
    unique_id_client : string
        name of unique id colomn in client geodataframe
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    import plotly.express as px
    
    data = ckdnearest(df_client, df_bvarta)
    df_bvarta['keterangan'] = np.where(df_bvarta[unique_id_bvarta].isin(
        data[data['keterangan']=='identical'][unique_id_bvarta].unique().tolist()),'identical','non-identical')
    df_client['keterangan'] = np.where(df_client[unique_id_client].isin(
        data[data['keterangan']=='identical'][unique_id_client].unique().tolist()),'identical','non-identical')
    df_bvarta['id'] = df_bvarta[unique_id_bvarta]
    df_client['id'] = df_client[unique_id_client]
    
    #visualisasi
    m = grid.explore(tiles = 'Cartodb Positron',
                     style_kwds = dict(color = 'grey',fill=False, weight = 0.5),
                     legend = False,
                     popup = False
                    )
    df_bvarta[df_bvarta['keterangan']=='non-identical'].explore(m=m,
                                                                color = 'darkgrey',
                                                               marker_kwds=dict(radius = 0.5))
    df_bvarta[df_bvarta['keterangan']=='identical'].rename(columns = {'keterangan':'Bvarta'}).explore(m=m,
                                                           column = 'Bvarta',
                                                            cmap = 'OrRd_r',
                                                           marker_kwds=dict(radius = 6, 
                                                                            )
                                                           )
    df_client[df_client['keterangan']=='identical'].rename(columns = {'keterangan':'Client'}).explore(m=m,
                                                           column = 'Client',
                                                            cmap = 'OrRd',
                                                            marker_kwds=dict(radius = 2, 
                                                                            )
                                                           )
    return m

def Visualisasi_Grid(grid_class, col_name='dominan'):
    """Fungsi untuk mendapatkan visualisasi identical POI
    Parameters
    ----------
    grid_class = geopandas.GeoDataFrame()
        geodataframe grid dominan from func grid_dominan
    col_name = string. Default 'dominan'. {'dominan','selisih','Total POI Bvarta','Total POI Client'}
        name of columns for classification
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    import plotly.express as px
    
    m = grid_class.explore(tiles = 'Cartodb Positron',
                     column = col_name,
                        cmap = 'OrRd',
                     legend = True,
                     tooltip = ['dominan','selisih','Total POI Bvarta','Total POI Client'])
    return m

def Correlation(grid_class):
    """Fungsi untuk mendapatkan scatter plot Korelasi POI Bvarta terhadap POI Client
    Parameters
    ----------
    grid = geopandas.GeoDataFrame()
        geodataframe grid dominan from func grid_dominan
    ----------
    """
    import numpy as np
    import plotly.express as px
    import plotly.graph_objs as go

    test = grid_class[['Total POI Bvarta','Total POI Client']]
    d = np.polyfit(test['Total POI Bvarta'],test['Total POI Client'],1)
    f = np.poly1d(d)
    test.insert(2,'Treg',f(test['Total POI Bvarta']))

    fig = px.scatter(test,
                     y='Total POI Client', 
                     x='Total POI Bvarta')
    fig.add_trace(go.Scatter(y=test['Treg'], 
                             x=test['Total POI Bvarta'],
                             mode="lines",
                             line=go.scatter.Line(color="red"),
                             showlegend=False))
    fig.show()
    
def Chart(df_bvarta, df_client,unique_id_bvarta, unique_id_client):
    """Fungsi untuk mendapatkan Chart Perbandingan POI Bvarta dan 
    POI Client berdasarkan kesamaan lokasi (identical non-identical)
    Parameters
    ----------
    df_client : geopandas.GeoDataFrame()
        geodataframe data Client
    df_bvarta : geopandas.GeoDataFrame()
        geodataframe data Bvarta
    unique_id_bvarta : string
        name of unique id colomn in bvarta geodataframe
    unique_id_client : string
        name of unique id colomn in client geodataframe
    ----------
    """
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import pandas as pd
    import numpy as np
    
    data = ckdnearest(df_client, df_bvarta)
    df_bvarta['keterangan'] = np.where(df_bvarta[unique_id_bvarta].isin(
        data[data['keterangan']=='identical'][unique_id_bvarta].unique().tolist()),'identical','non-identical')
    df_client['keterangan'] = np.where(df_client[unique_id_client].isin(
        data[data['keterangan']=='identical'][unique_id_client].unique().tolist()),'identical','non-identical')
    df_bvarta['id'] = df_bvarta[unique_id_bvarta]
    df_client['id'] = df_client[unique_id_client]
    
    df_bvarta['data'] = 'Bvarta'
    df_client['data'] = 'Client'
    merge = pd.concat([df_bvarta[['id','geometry','keterangan','data']],
                     df_client[['id','geometry','keterangan','data']]])
    
    #Chart1
    group = merge.groupby(['data'])['id'].nunique().reset_index()
    labels = group.data.unique().tolist()
    values = group['id'].tolist()

    # pull is given as a fraction of the pie radius
    fig = go.Figure(data=[go.Pie(labels = labels, 
                                 values = values, 
                                 pull = [0, 0.2],
                                ),
                         ])
    fig.update_layout(
        title_text="Perbandingan Data Bvarta terhadap Data Client")
    fig.show()
    
    #Chart2
    group = merge.groupby(['keterangan','data'])['id'].nunique().reset_index()
    

    labels = group.keterangan.unique().tolist()

    # Create subplots: use 'domain' type for Pie subplot
    fig = make_subplots(rows=1, 
                        cols=2, 
                        specs=[[{'type':'domain'}, {'type':'domain'}]])
    fig.add_trace(go.Pie(labels=labels, 
                         values=group[group['data']=='Bvarta']['id'].tolist(), 
                         name="Bvarta",
                         textinfo='value+percent'),
                  1, 1)
    fig.add_trace(go.Pie(labels=labels, 
                         values=group[group['data']=='Client']['id'].tolist(), 
                         name="Client",
                         textinfo='value+percent'),
                  1, 2)

    # Use `hole` to create a donut-like pie chart
    fig.update_traces(hole=.4, 
                      hoverinfo="label+percent+name")

    fig.update_layout(
        title_text="Perbandingan data Identical Non-Identical",
        # Add annotations in the center of the donut pies.
        annotations=[dict(text='Bvarta', x=0.18, y=0.5, font_size=20, showarrow=False),
                     dict(text='Client', x=0.82, y=0.5, font_size=20, showarrow=False)])
    fig.show()